# Работа с GigaChat Vision
В этом ноутбуке мы рассмотрим работу с GigaChat Vision

Наши действия:
1. Загружаем фото на S3 хранилище GigaChat
2. Просим GigaChat проанализировать фотографии, выделить сущности на них и написать описание к фото
3. Выдать ответ в виде JSON, после чего преобразуем этот JSON в Pydantic модель

Тестировать будем на этих фото
![фото 1](cat.jpg)
![фото 2](sea.jpg)


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_gigachat.chat_models.gigachat import GigaChat
llm = GigaChat(
    credentials="MTEwMzY1YmEtMzYzMy00YWQ1LThmMTQtNWEzODM0NzUwN2IwOjU2ZDlkOGY1LWU0MDUtNDQ2Ni1hNTQyLWU5N2M4MzJmY2FmOA==",
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro-preview",
    base_url="https://gigachat-preview.devices.sberbank.ru/api/v1"
)

In [2]:
from typing import List, Optional
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
from langchain_core.runnables import RunnableLambda, RunnableParallel


class Photo(BaseModel):
    """Информация о фото"""

    content: str = Field(..., description="Что изображено на фото? 1-3 слова")
    description: str = Field(..., description="Опиши детальнее фото")


# Set up a parser
parser = PydanticOutputParser(pydantic_object=Photo)


def _get_messages_from_url(url: str):
    return {
        "history": [
            HumanMessage(content="", additional_kwargs={"attachments": [url]}),
        ]
    }


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            #"Определи содержимое на фото. Ответь на запрос пользователя в формате JSON. Schema Information: \n{format_instructions}",
            "Определи содержимое на фото.",
        ),
        MessagesPlaceholder("history"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = RunnableLambda(_get_messages_from_url) | prompt | llm #| parser

В итоге мы создали LCEL цепочку, в которую мы можем передавать id загруженных файлов и в ответ получать Pydantic модели

In [4]:
# Пример работы

file1 = llm.upload_file(open("chart1.jpg", "rb"))
file2 = llm.upload_file(open("chart2.jpg", "rb"))
file3 = llm.upload_file(open("chart3.jpg", "rb"))
chain.batch([file1.id_, file2.id_, file3.id_])


[AIMessage(content='Mobile Gaming, PC/Mac Gaming, Home Consoles, Handheld Consoles', additional_kwargs={}, response_metadata={'token_usage': Usage(prompt_tokens=1300, completion_tokens=20, total_tokens=1320), 'model_name': 'GigaChat-Pro-preview:1.0.26.15', 'finish_reason': 'stop'}, id='run-6dcda024-1784-4b90-834b-d27f51440678-0'),
 AIMessage(content="The chart displays data on consumer spending shares on games across different platforms and regions for two periods: Q1 2023 versus Q1 2024, and Q1 2023 to Q1 2024. The legend at the bottom indicates that each color represents a specific region: North America (blue), Western Europe (orange), Asia-Pacific (green), and Rest of World (purple).\n\nThe top part of the chart shows mobile gaming spending share trends by region over the given time period. In Q1 2023, North America had a 5.4% share, which increased to 6.2% in Q1 2024. Similarly, Western Europe's share went from 9.4% to 10.1%. Asia-Pacific saw an increase from 30.4% to 34.0%, while 

In [ ]:
"""
file1 = llm.upload_file(open("chart1.jpg", "rb"))
chain.batch([file1.id_])
"""